In [ ]:
from scipy.stats import multivariate_normal
import cv2
from scipy import io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.ndimage import label,map_coordinates,convolve1d, rank_filter
from scipy.stats import multivariate_normal
from numpy import unravel_index
import math
import glob
from matplotlib import rcParams
import base64
from numpy.linalg import norm
import os
import sys
%matplotlib inline



In [ ]:
#1.1

mat = io.loadmat('skinSamplesRGB.mat')
skin_YCbCr = cv2.cvtColor(mat['skinSamplesRGB'], cv2.COLOR_RGB2YCrCb)
Cb = skin_YCbCr[:,:,1] 
Cr = skin_YCbCr[:,:,2]
mu = [np.mean(Cb),np.mean(Cr)]
cov = np.array(np.cov(Cb.flatten(), Cr.flatten()))
print("mu = ", mu)
print("cov = ", cov)


In [ ]:
global ct
ct=0
#plt.savefig(str(ct))
#ct=ct+1

In [ ]:
#boundingBox Function

def bounds(I,mu,cov):
    global ct
    IRGB = cv2.cvtColor(I, cv2.COLOR_BGR2RGB)
    IYCbCr = cv2.cvtColor(IRGB, cv2.COLOR_RGB2YCrCb)
    Cb = IYCbCr[:,:,1]
    Cr = IYCbCr[:,:,2]

    x= np.array( [Cb, Cr] ).T
    P = multivariate_normal.pdf( x , mu, cov) 
    P = P.T

    _, skin = cv2.threshold(P,0.0001, 0.25, cv2.THRESH_BINARY) #den paei to 0.05 pou leei gia katw value

    kernel = np.ones((5,5),np.uint8) #changed
    opening = cv2.morphologyEx(skin, cv2.MORPH_OPEN, kernel)
    kernel = np.ones((10,10),np.uint8) 
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)

    labeled_array, num_features=label(closing)
    coords=[]
    
    

    fig1, ax = plt.subplots(1)
    colors=['r','g','b']
    j=0
    for i in range(num_features):
        x,y = np.where(labeled_array == i+1)
        xmax=np.max(x)
        xmin=np.min(x)
        ymax=np.max(y)
        ymin=np.min(y)
        width = xmax-xmin
        height = ymax - ymin
        if(height>10 or width>10):
            coords.append([ymin,xmin,height,width]) #i np.where ta vgaze anapoda ta x,y
            ax.add_patch(patches.Rectangle((ymin,xmin),height,width, edgecolor=colors[j], facecolor="none",linewidth=2))
            j=j+1
    ax.imshow(IRGB)
    ax.set_title('Skin areas')
    title='part1-results/'
    name=str(ct)
    plt.savefig(title+name)
    ct=ct+1
         
    
    
    
    
    fig2, ax2 = plt.subplots(1, 3, figsize = (10,30))
    ax2[0].imshow(IRGB)
    ax2[1].imshow(IYCbCr)
    ax2[2].imshow(closing)
    ax2[0].set_title("Original Image")
    ax2[1].set_title("YCbCr Image")
    ax2[2].set_title("Skin image after closing")
    title='part1-results/'
    name=str(ct)
    plt.savefig(title+name)
    ct=ct+1
         
    print("Head Coords: ", coords[0])
    print("Left Hand Coords: ", coords[1])
    print("Right Hand Coords:", coords[2])
    
    
    return coords





In [ ]:
img = cv2.imread("part1 - GreekSignLanguage/1.png")
bounds(img,mu,cov)


In [ ]:
x, y = np.mgrid[140:170:1, 90:130:1]
pos = np.dstack((x, y))
rv = multivariate_normal(mu, cov)
rcParams['figure.figsize'] = [8,5]
plt.contourf(x, y, rv.pdf(pos))
plt.title('Gaussian Probability Density Function')
plt.xlabel('Cr Channel')
plt.ylabel('Cb Channel')
#plt.show()
title='part1-results/'
name=str(ct)
plt.savefig(title+name)
ct=ct+1

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(25,10))
ax = fig.add_subplot(1, 2, 1, projection='3d')
ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.plot_wireframe(x, y, rv.pdf(pos)/np.max(rv.pdf(pos)), color='black')#)cmap=cm.PuOr)
plt.title('Gaussian Distribution in 3D Space', y=1)
plt.xlabel('Cr Channel')
plt.ylabel('Cb Channel')
ax.set_zlabel('Normalized Probability')
title='part1-results/'
name=str(ct)
plt.savefig(title+name)
ct=ct+1
plt.show()



In [ ]:
#1.2.1

#### lukas kanade ####

def lk(I1,I2,r0,e):
    global ct
    
    #initialize
    I1 = cv2.cvtColor(I1, cv2.COLOR_BGR2RGB)
    I2 = cv2.cvtColor(I2, cv2.COLOR_BGR2RGB)
    
    I1gray = cv2.cvtColor(I1, cv2.COLOR_RGB2GRAY)
    I2gray = cv2.cvtColor(I2, cv2.COLOR_RGB2GRAY)
    
    iters=100
    p0 = cv2.goodFeaturesToTrack(I1gray, mask = None, **feature_params)
    
    dx_0=np.zeros_like(I1gray)
    dy_0=np.zeros_like(I1gray)
    
    #img to [0,1]
    I1gray = I1gray.astype(np.float64)/255
    I2gray = I2gray.astype(np.float64)/255

    dx_i = dx_0.astype(np.float64)
    dy_i = dy_0.astype(np.float64) 


    #filter
    n = int(2*np.ceil(3*r0)+1)  
    gauss1D = cv2.getGaussianKernel(n, r0) 
    gauss2D = gauss1D @ gauss1D.T  

    #grad_of_img
    dy_Iprev, dx_Iprev = np.gradient(I1gray)

    #meshgrid
    x_0,y_0 = np.meshgrid(np.arange(0,I1.shape[1]),np.arange(0,I1.shape[0]))


    for i in range(iters):

        ###### Calculate I(x+dx) by interpolation ##########
        Iprev = map_coordinates(I1gray,[np.ravel(y_0 + dy_i),np.ravel(x_0 + dx_i)], order=1).reshape(I2gray.shape)
        A1 = map_coordinates(dx_Iprev,[np.ravel(y_0 + dy_i), np.ravel(x_0 + dx_i)], order=1).reshape(I1gray.shape)
        A2 = map_coordinates(dy_Iprev,[np.ravel(y_0 + dy_i), np.ravel(x_0 + dx_i)], order=1).reshape(I1gray.shape)
        E= I2gray - Iprev

        #mathematical processing..
        a11 = cv2.filter2D(A1**2,-1,gauss2D)+e
        a12 = cv2.filter2D(A1*A2,-1,gauss2D)
        a21 = a12
        a22 = cv2.filter2D(A2**2,-1,gauss2D)+e

        b1 = cv2.filter2D(A1*E,-1,gauss2D)
        b2 = cv2.filter2D(A2*E,-1,gauss2D)

        det= (a11*a22 - a12*a21) 

        ux = (a22*b1 - a12*b2) / det
        uy = (a11*b2 - a21*b1) / det

        if p0 is not None:
                for x,y in p0[:,0,:].astype(int):
                    dx_i[y][x] = dx_i[y][x] + ux[y][x]
                    dy_i[y][x] = dy_i[y][x] + uy[y][x]

    return dx_i, dy_i
                
    
            





    
    
    







In [ ]:
##### part 2 ######
def speed(dx,dy,thr):
    global ct
    d=dx**2+dy**2
    cond = (d >= thr*np.max(d)).astype(np.uint8)

    dx_new=cond*dx
    dy_new=cond*dy

    dx_mean = np.sum(dx_new)/np.sum(cond)
    dy_mean = np.sum(dy_new)/np.sum(cond)

    return dx_mean, dy_mean



####################

In [ ]:
def multi_lk(I1, I2, r0, e, scales): #apli ilopoihsh
    global ct
    
    I1 = cv2.cvtColor(I1, cv2.COLOR_BGR2RGB)
    I2 = cv2.cvtColor(I2, cv2.COLOR_BGR2RGB)
    
    I1s, I2s = [], []
    I1s.append(I1)
    I2s.append(I2)
    for i in range(scales): #scale_down
        Gr = cv2.getGaussianKernel(3, r0) # 3-pixel Gaussian kernel
        # Lowpass Filter to avoid aliasing
        I1 = cv2.filter2D(I1, -1, Gr)
        I2 = cv2.filter2D(I2, -1, Gr)
        # Downsample and save
        I1 = cv2.pyrDown(I1)
        I2 = cv2.pyrDown(I2)
        I1s.append(I1)
        I2s.append(I2)
        
    I1gray = cv2.cvtColor(I1, cv2.COLOR_RGB2GRAY)
    I2gray = cv2.cvtColor(I2, cv2.COLOR_RGB2GRAY)
    
    dx_0 = np.zeros_like(I1gray)
    dy_0 = np.zeros_like(I2gray)
    
    for i in range(scales):
        I1gray = cv2.cvtColor(I1, cv2.COLOR_RGB2GRAY)
        p0 = cv2.goodFeaturesToTrack(I1gray, mask = None, **feature_params)
        dx_0,dy_0 = lk(I1,I2,r0,e)
        if (i == scales-1):
            break
        else:
            I1 = I1s[scales-2-i]
            I2 = I2s[scales-2-i]
            dx_0 = 2*cv2.resize(dx_0,( int(np.ceil(I1.shape[1])) , int(np.ceil(I1.shape[0] ))))
            dy_0 = 2*cv2.resize(dy_0,( int(np.ceil(I1.shape[1])) , int(np.ceil(I1.shape[0] ))))
    return dx_0, dy_0

In [ ]:
#initial parameters and images

feature_params = dict( maxCorners = 30,
                       qualityLevel = 0.2,
                       minDistance = 2,
                       blockSize = 5)

I1 = cv2.imread('part1 - GreekSignLanguage/1.png')
I2 = cv2.imread('part1 - GreekSignLanguage/5.png')

I1 = cv2.cvtColor(I1, cv2.COLOR_BGR2RGB)
I2 = cv2.cvtColor(I2, cv2.COLOR_BGR2RGB)


fig, ax = plt.subplots(1, 2, figsize = (10,30))
ax[0].imshow(I1)
ax[1].imshow(I2)
ax[0].set_title("Original Image I1")
ax[1].set_title("Moved Image I2")
title='part1-results/'
name=str(ct)
plt.savefig(title+name)
ct=ct+1
plt.show()



In [ ]:
#plot l-k flow for I1,I2
dx_arx, dy_arx = lk(I1,I2,5,0.1)
plt.gca().invert_yaxis()
plt.quiver(-dx_arx,-dy_arx,angles='xy',scale=100)
plt.title("Optical Flow with Lukas-Kanade")
title='part1-results/'
name=str(ct)
plt.savefig(title+name)
ct=ct+1
plt.show()


In [ ]:
#plot multi l-k flow for I1,I2
dx_sc, dy_sc = multi_lk(I1,I2,5,0.1,8)
plt.gca().invert_yaxis()
plt.quiver(-dx_sc,-dy_sc,angles='xy',scale=100)
plt.title("Optical Flow with Multiscale Lukas-Kanade")
title='part1-results/'
name=str(ct)
plt.savefig(title+name)
ct=ct+1
plt.show()



    



In [ ]:
print(dx_arx==dx_sc)
print(dy_arx==dy_sc)

In [ ]:
#compute total metatopisi
dx_mean_arx, dy_mean_arx= speed(dx_arx,dy_arx,0.5)
print("dx_mean with lk = ", dx_mean_arx)
print("dy_mean with lk = ",dy_mean_arx)

dx_mean_sc, dy_mean_sc= speed(dx_sc,dy_sc,0.5)
print("dx_mean with Multi_scale_lk = ", dx_mean_sc)
print("dy_mean with Multi_scale_lk = ",dy_mean_sc)




In [ ]:
#multivariable analysis  
epsilons=[0.01, 0.025, 0.05, 0.075, 0.1]
r0s=[1,2,3,4,5]
scales=[1,2,3,4,5]


for i in range(5): #1
    dx, dy = lk(I1,I2,5,epsilons[i])
    plt.gca().invert_yaxis()
    plt.quiver(-dx,-dy,angles='xy',scale=100)
    e0=float(epsilons[i])
    r0=5
    plt.title("Optical Flow with Lukas-Kanade for e =  %.3f" % e0 + " and ρ = %.1f" %r0)
    title='part1-results/'
    name=str(ct)
    plt.savefig(title+name)
    ct=ct+1
    plt.show()

    

In [ ]:
for i in range(5): #1 
    dx, dy = lk(I1,I2,5,epsilons[i]) 
    plt.gca().invert_yaxis() 
    plt.quiver(-dx,-dy,angles='xy',scale=100) 
    e0=float(epsilons[i]) 
    r0=5 
    plt.title("Optical Flow with Lukas-Kanade for e = %.3f" % e0 + " and ρ = %.1f" %r0) 
    title='part1-results/' 
    name=str(ct) 
    plt.savefig(title+name) 
    ct=ct+1 
    plt.show()


In [ ]:
for i in range(5): #2
    dx, dy = lk(I1,I2,r0s[i],0.1)
    plt.gca().invert_yaxis()
    plt.quiver(-dx,-dy,angles='xy',scale=100)
    e0=0.1
    r0=r0s[i]
    plt.title("Optical Flow with Lukas-Kanade for e = %.3f" % e0 + " and ρ = %.1f" %r0)
    title='part1-results/'
    name=str(ct)
    plt.savefig(title+name)
    ct=ct+1
    plt.show()
    

In [ ]:
for i in range(5): #3
    dx, dy = lk(I1,I2,5,epsilons[i])
    dx_mean,dy_mean=speed(dx,dy,0.5)
    e0=float(epsilons[i])
    r0=5
    print("e = %.3f" %e0+" ,ρ = %.3f" %r0+" ,dx = %.3f" %dx_mean+" ,dy = %.3f" %dy_mean)

In [ ]:
for i in range(5): #4
    dx, dy = lk(I1,I2,r0s[i],0.1)
    dx_mean,dy_mean=speed(dx,dy,0.5)
    e0=0.1
    r0=r0s[i]
    print("e = %.3f" %e0+" ,ρ = %.3f" %r0+" ,dx = %.3f" %dx_mean+" ,dy = %.3f" %dy_mean)

In [ ]:
for i in range(5): #5
    dx, dy = multi_lk(I1,I2,5,epsilons[i],5)
    plt.gca().invert_yaxis()
    plt.quiver(-dx,-dy,angles='xy',scale=100)
    e0=float(epsilons[i])
    r0=5
    sc=5
    plt.title("Optical Flow with Multiscale Lukas-Kanade for e =  %.3f" % e0 + " ,ρ = %.1f" %r0+ " and scales = %.3f" %sc)
    title='part1-results/'
    name=str(ct)
    plt.savefig(title+name)
    ct=ct+1
    plt.show()

In [ ]:
for i in range(5): #6
    dx, dy = multi_lk(I1,I2,r0s[i],0.1,5)
    plt.gca().invert_yaxis()
    plt.quiver(-dx,-dy,angles='xy',scale=100)
    e0=0.1
    r0=r0s[i]
    sc=5
    plt.title("Optical Flow with Multiscale Lukas-Kanade for e =  %.3f" % e0 + " ,ρ = %.1f" %r0+ " and scales = %.3f" %sc)
    title='part1-results/'
    name=str(ct)
    plt.savefig(title+name)
    ct=ct+1
    plt.show()

In [ ]:
for i in range(5): #7
    dx, dy = multi_lk(I1,I2,5,0.1,scales[i])
    plt.gca().invert_yaxis()
    plt.quiver(-dx,-dy,angles='xy',scale=100)
    e0=0.1
    r0=5
    sc=scales[i]
    plt.title("Optical Flow with Multiscale Lukas-Kanade for e =  %.3f" % e0 + " ,ρ = %.1f" %r0+ " and scales = %.3f" %sc)
    title='part1-results/'
    name=str(ct)
    plt.savefig(title+name)
    ct=ct+1
    plt.show()

In [ ]:
for i in range(5): #8
    dx, dy = multi_lk(I1,I2,5,epsilons[i],5)
    dx_mean,dy_mean=speed(dx,dy,0.5)
    e0=epsilons[i]
    r0=5
    sc=5
    print("e = %.3f" %e0+" ,ρ = %.3f" %r0+ " ,scales = %.3f"%sc+ " ,dx = %.3f" %dx_mean+" ,dy = %.3f" %dy_mean)

In [ ]:
for i in range(5): #9
    dx, dy = multi_lk(I1,I2,r0s[i],0.1,5)
    dx_mean,dy_mean=speed(dx,dy,0.5)
    e0=0.1
    r0=r0s[i]
    sc=5
    print("e = %.3f" %e0+" ,ρ = %.3f" %r0+ " ,scales = %.3f"%sc+ " ,dx = %.3f" %dx_mean+" ,dy = %.3f" %dy_mean)

In [ ]:
for i in range(5): #10
    dx, dy = multi_lk(I1,I2,5,0.1,scales[i])
    dx_mean,dy_mean=speed(dx,dy,0.5)
    e0=0.1
    r0=5
    sc=scales[i]
    print("e = %.3f" %e0+" ,ρ = %.3f" %r0+ " ,scales = %.3f"%sc+ " ,dx = %.3f" %dx_mean+" ,dy = %.3f" %dy_mean)

In [ ]:
thrs=[0.001,0.1,0.2,0.5,0.75]
for i in range(5): #12
    dx, dy = multi_lk(I1,I2,5,0.1,5)
    dx_mean,dy_mean=speed(dx,dy,thrs[i])
    e0=0.1
    r0=5
    sc=5
    thr=thrs[i]
    print("e = %.3f" %e0+" ,ρ = %.3f" %r0+ " ,scales = %.3f"%sc+ " ,threshold = %.3f" %thr+ " ,dx = %.3f" %dx_mean+" ,dy = %.3f" %dy_mean)

In [ ]:
thrs=[0.001,0.1,0.2,0.5,0.75]
for i in range(5): #11
    dx, dy = lk(I1,I2,5,0.1)
    dx_mean,dy_mean=speed(dx,dy,thrs[i])
    e0=0.1
    r0=5
    thr=thrs[i]
    print("e = %.3f" %e0+" ,ρ = %.3f" %r0+" ,threshold = %.3f" %thr+ " ,dx = %.3f" %dx_mean+" ,dy = %.3f" %dy_mean)

In [ ]:
def motion_traction(part,algo):
    global ct
    #part0->left
    #part1->right
    #part2->head
    #algo0->0
    if(part==0 and algo==0):
        titlee="Motion Traction for left hand with Lukas Kanade"
    if(part==0 and algo==1):
        titlee="Motion Traction for left hand with Multi-Scale Lukas Kanade"
    if(part==1 and algo==0):
        titlee="Motion Traction for right hand with Lukas Kanade"
    if(part==1 and algo==1):
        titlee="Motion Traction for right hand with Multi-scale Lukas Kanade"
    if(part==2 and algo==0):
        titlee="Motion Traction for head with Lukas Kanade"
    if(part==2 and algo==1):
        titlee="Motion Traction for head with Multi-scale Lukas Kanade"
        
    if(part==0):
        x=47 #for left hand
        y=243 
        height=71 - 3
        width=66 - 3

    if(part==2):
        x = 138 #for head
        y = 88
        width = 73 - 2
        height = 123 - 2


    #Parameters to track the right hand
    if(part==1):
        x = 162
        y = 264
        width = 83 - 3
        height = 48 - 3


    for i in range(1,65):
        path1 = 'part1 - GreekSignLanguage/' + str(i) + '.png'
        I1 = cv2.imread(path1)

        path2 = 'part1 - GreekSignLanguage/' + str(i+1) + '.png'
        I2 = cv2.imread(path2)
        
        I1 = cv2.cvtColor(I1, cv2.COLOR_BGR2RGB)
        I2 = cv2.cvtColor(I2, cv2.COLOR_BGR2RGB)
        
        

        det1 = I1[y:(y+height),x:(x+width)]
        det2 = I2[y:(y+height),x:(x+width)]
        
        
        if(algo==0): d_x, d_y = lk(det1, det2, 7, 0.1)
        if(algo==1): d_x, d_y = multi_lk(det1, det2, 7, 0.1, 5)
        

        displ_x,displ_y = speed(d_x, d_y,0.5)
        displ_x = int(displ_x)
        displ_y = int(displ_y)

        #print(displ_x,displ_y)

        plt.figure()
        cv2.rectangle(I2, (x, y), (x+width, y+height), (255,0,0) ,2)
        plt.imshow(I2)
        plt.title(titlee+",frame " +str(i))
        
        title='part1-results/'
        name=str(ct)
        plt.savefig(title+name)
        ct=ct+1
        plt.show()
        x = x - (displ_x)
        y = y - (displ_y)
        if(displ_x == 0 and displ_y == 0 and abs(height-width)==35):
            #x = x-(displ_x-10)
            y = y-(displ_y-10)
            
        #if(displ_x == 0 and displ_y == 0 and abs(height-width)==50):
         #   x = x-(displ_x+10)
          #  y = y-(displ_y+10)
        


In [ ]:
motion_traction(0,0)

In [ ]:
motion_traction(1,0)

In [ ]:
motion_traction(2,0)

In [ ]:
motion_traction(0,1)

In [ ]:
motion_traction(1,1)

In [ ]:
motion_traction(2,1)